In [1]:
import re
from bs4 import BeautifulSoup
import requests
import html5lib
import json

In [2]:
url = 'https://www.cna.com.tw/list/aspt.aspx'
res = requests.get(url)
soup = BeautifulSoup(res.text,'html5lib')
n=0
labels=[]
while n < len(soup.select_one('#pnProductNavContents > ul').findAll('li')):
    try :
        lab = soup.select_one('#pnProductNavContents > ul').findAll('li')[n].find('a',{'class':'first-level'}).text
        url =soup.select_one('#pnProductNavContents > ul').findAll('li')[n].find('a',{'class':'first-level'})['href']
    except:(TypeError,IndentationError)
    labels.append((url,lab))
    n += 1
labels = list(set(labels))
labels.remove(('http://cnavideo.cna.com.tw', '影音'))
labels.remove(('/list/aall.aspx', '即時'))
labels.append(('https://www.cna.com.tw/list/aall.aspx', '即時'))
#labels.remove(('https://www.cna.com.tw/keyword/九合一.aspx', '九合一'))
labels.remove(('https://www.cna.com.tw/list/sp.aspx', '專題'))
labels.remove(('https://www.cna.com.tw/list/aall.aspx', '即時'))

labs = []
for item in labels:
    item = [item[0],item[1]]

    item[0] = re.search(r'list\/\w+',item[0]).group()[5:]
    labs.append(item)

In [3]:
def payload(item):
    payload={
        'id': '1056893641102242',
    'ev': 'Microdata',
    'dl': item[0],
    'rl': 'https://www.cna.com.tw/',
    'if': 'false',
    'ts': '1541087113735',
    'cd[Schema.org]': [],
    #'cd[OpenGraph]': {"og:url":"https://www.cna.com.tw/list/ahel.aspx","og:title":"生活 | 中央社 CNA","og:description":"想知道統一發票、大樂透、威力彩開獎號碼、各種醫療保健、交通氣象、颱風動態等生活資訊，及升學考試、教育政策、課綱改革等文教訊息，通通都在中央社生活新聞。","og:image:height":"200","og:image:width":"200","og:image":"https://img5.cna.com.tw/www/images/pic_fb.jpg","og:type":"article","article:author":"https://www.facebook.com/cnanewstaiwan","article:publisher":"https://www.facebook.com/cnanewstaiwan"}
    #'cd[Meta]': {"title":"\n\t生活 | 中央社 CNA\n","meta:description":"想知道統一發票、大樂透、威力彩開獎號碼、各種醫療保健、交通氣象、颱風動態等生活資訊，及升學考試、教育政策、課綱改革等文教訊息，通通都在中央社生活新聞。","meta:keywords":"News, 新聞, 即時新聞, 中央社"}
    'cd[DataLayer]': [],
    'cd[JSON-LD]': [],
    'sw': '1280',
    'sh': '5000',#'720',
    'v': '2.8.30',
    'r': 'stable',
    'a': 'tmgoogletagmanager',
    'ec': '1',
    'o': '30',
    'fbp': 'fb.2.1541074934894.620990608',
    'it': '1541087112886',
    'coo': 'false',
    'es': 'automatic',
    }
    return payload

In [4]:
labs_href=[]
for item in labs:
    soups=[]
    n=1
    while n < 13:
        url = 'https://www.cna.com.tw/cna2018api/api/simplelist/categorycode/'+item[0]+'/pageidx/'+str(n)
        res = requests.get(url)
        soup = BeautifulSoup(res.text,'html5lib')
        body = soup.findAll('body')[0].getText()
        PageUrl = re.findall(r'\"PageUrl\"\:\S[^\,]+',body)
        urls = [txt[11:-1] for txt in PageUrl]
        soups.extend(urls)
        n += 1
        if len(urls) < 20:
            break
    labs_href.append([soups,item[1]])

In [5]:
news={}
newslist=[]
for lab in labs_href:
    lab_name = lab[1]
    lab_urls = lab[0]
    n = 0
    for url in lab_urls:
        res = requests.get(url)
        soup = BeautifulSoup(res.text,'html5lib')
        txt = ""
        try:
            title = soup.find_all('div',{'class':'centralContent'})[0].find_all('h1')[0].getText()
            for s in soup.find_all('div',{'class':'paragraph'})[0].findAll('p'):
                txt=txt+s.getText()
        except:IndexError
        research = re.search(r'[\）\)\：\:\》](.+[\。\!\?\s])+',txt)
        if research == None or len(research.group())<2:
            print(txt)
        else:
            txt = re.search(r'[\）\)\：\:\》](.+[\。\!\?\s])+',txt).group()[1:]

        page = [[title,txt],lab_name]
        newslist.append(page)
        news[(title,txt)]=lab_name
        n += 1
        if n % 20 == 0:
            print((lab_name,n),end=",")

('產經', 20),('產經', 40),('產經', 60),('產經', 80),('產經', 100),('政治', 20),('政治', 40),('政治', 60),('政治', 80),('政治', 100),('地方', 20),('地方', 40),('地方', 60),('地方', 80),('地方', 100),('證券', 20),('證券', 40),('證券', 60),('證券', 80),('證券', 100),('娛樂', 20),('娛樂', 40),('娛樂', 60),('娛樂', 80),('娛樂', 100),('國際', 20),('國際', 40),('國際', 60),('國際', 80),('國際', 100),('兩岸', 20),('兩岸', 40),中央社駐上海特派員陳家倫/1月10日現年92歲的上海海關大鐘近來開始誤點，去年8月還一度中暑暈厥了好幾小時。衰老的大鐘預計於今年接受英國原廠「健檢」，不論結果是搶救或無痛地走向終點，都將轟動上海灘。上海海關大樓是外灘知名歷史建築，頂部的大鐘與倫敦大笨鐘是「姊妹」，由英國同一家公司生產。據稱，上海海關大鐘當年總造價高達5000多兩白銀，於1927年8月從倫敦運到上海，原包裝木箱連同大鐘重達6.25噸。1928年元旦凌晨1時，海關大鐘敲響了第一聲，從此以後，鐘聲就日以繼夜地陪伴著上海人。對於早年生活並不富裕的中國民眾而言，別說手上戴個表，連在家掛個時鐘都是奢侈，噹噹響的鐘聲因而成了市民珍視的「公共財」。「以前再遠都聽得到呦」，62歲的上海徐姓民眾從小在靜安寺附近長大，住家距離外灘約5公里遠。他回憶，學齡前就對海關大鐘的鐘聲有印象，但受到都市化的高樓阻絕、汽車雜音干擾，老家現已聽不到鐘聲。至於原先住在外灘一帶的居民，因都市計畫與拆遷的關係，也從海關大鐘的「搖滾區」撤出，接替而上的是大批境外觀光客，連帶將海關大鐘推向世界，鐘聲敲進了更多人的耳裡，並迴盪在老上海的記憶中。享受世人矚目的同時，邁入92歲的海關大鐘也面臨著歲月的考驗，自去年起，就開始不太準時，常有熱心民眾去電通報「大鐘時間錯了」；上海的夏季高溫則是另一道試煉，去年8月一個攝氏40度烤曬的午後，老鐘昏了過去。守鐘人魏云寺表示，海關大鐘自去年暑假開始，每到下午就走走停停，他當時就想，位於10樓的鐘樓溫度往往比外頭多

In [6]:
import jieba
import nltk

In [7]:
stopwords=[]
with open('../30/txt/stopwords_ch.txt', 'r', encoding='UTF-8') as file:
    for data in file.readlines():
        data = data.strip()
        stopwords.append(data)

In [8]:
lab_tags ={}
prelab = ''
books = list(news.items())
for book in books:
    for lab in ['生活','運動','政治','科技','國際','兩岸','產經','社會','地方','娛樂','證券','文化']:
        if book[1]==lab:
            if prelab != lab:
                txt=''
            txt = txt+book[0][1]
            lab_tags[book[1]] = txt
            prelab = lab

In [9]:
filterwords=[
    '年','月','中央社','記者','編輯','譯者','日電'
    ,'\n',' ','「','」','『','』'
    ,'今天','說','表示','年','月','日','後','前','時','應','中','今年'
    ,'兩','下','人','最','次','再','先','盡量'
]

In [10]:
articles = []
for news in newslist:
    articles.append((news[0][1],news[1]))
    

In [11]:
books = []


for n in range(len(articles)):

    txt_dic = {}
    txt = articles[n]
    txt_dic[txt[0]] = txt[1]
    books.append(txt_dic)


In [332]:
def get_tags(books=books):
    tags = {}  
    labs = list(set([v for b in books for v in b.values()]))
    for lab in labs:
        txt = ""
        for book in books:
            if list(book.values())[0] == lab:
                txt += list(book.keys())[0]
        tags[lab] = txt
    return tags

In [333]:
txt_tags = get_tags(books)

In [334]:
allrawword =""
for v in txt_tags.values():
    allrawword += v

In [335]:
def get_tokens(txt=""):
    seg_list = jieba.cut(txt, cut_all=False)
    txt = ",".join(seg_list)
    tokens = txt.split(',')
    tokens = [re.match(r'\w+',w).group() for w in tokens if (w not in stopwords and w not in filterwords) and re.match(r'\w+',w) != None]
    lab_alltwowords = list(nltk.bigrams(tokens))
    tokens.extend([str(i[0])+str(i[1]) for i in lab_alltwowords])
    return tokens

In [336]:
def get_fwords(txt,top=None):
    tokens = get_tokens(txt)
    return Counter(tokens).most_common(top)

In [337]:
def get_fwords_tags(top=None):
    
    fwords_tags = {}
    for i in list(txt_tags.items()):
        fwords_tags[i[0]]=get_fwords(i[1],top)
    return fwords_tags

In [338]:
fwords_tags_items = get_fwords_tags(300)

In [339]:
fwords_tags_items['兩岸']

[('中國', 397),
 ('台灣', 188),
 ('香港', 186),
 ('北京', 152),
 ('報導', 131),
 ('國家', 115),
 ('大陸', 96),
 ('指出', 86),
 ('一國兩制', 84),
 ('共識', 83),
 ('已', 81),
 ('政府', 79),
 ('元', 75),
 ('2018', 74),
 ('習近平', 73),
 ('經濟', 73),
 ('去年', 69),
 ('中共', 69),
 ('兩岸', 65),
 ('九二', 64),
 ('工作', 64),
 ('九二共識', 64),
 ('市場', 63),
 ('名', 62),
 ('上海', 60),
 ('提出', 58),
 ('12', 56),
 ('認為', 55),
 ('民主', 54),
 ('新', 53),
 ('進行', 53),
 ('美國', 52),
 ('總統', 52),
 ('問題', 51),
 ('會議', 50),
 ('中國大陸', 50),
 ('方面', 49),
 ('發展', 49),
 ('中央', 48),
 ('公司', 48),
 ('相關', 47),
 ('人員', 47),
 ('城市', 45),
 ('關係', 44),
 ('10', 44),
 ('接受', 43),
 ('大學', 42),
 ('項', 41),
 ('政策', 41),
 ('委員會', 41),
 ('發表', 40),
 ('中心', 40),
 ('台', 40),
 ('包括', 40),
 ('統一', 40),
 ('發布', 40),
 ('聯合', 40),
 ('民眾', 39),
 ('社會', 39),
 ('未', 38),
 ('新聞', 38),
 ('媒體', 38),
 ('要求', 37),
 ('國際', 36),
 ('當局', 36),
 ('管理', 36),
 ('政治', 35),
 ('蔡', 34),
 ('目前', 34),
 ('曾', 34),
 ('蔡總統', 34),
 ('自由', 33),
 ('人士', 33),
 ('服務', 33),
 ('北京方面', 33),
 ('支持', 32),
 (

In [340]:
vocabulary_list = [i[0] for i in get_fwords(allrawword,300)]

In [341]:
vocabulary_list

['台灣',
 '元',
 '中國',
 '去年',
 '已',
 '美國',
 '億',
 '指出',
 '新',
 '政府',
 '文化',
 '12',
 '億元',
 '10',
 '分',
 '國家',
 '公司',
 '名',
 '報導',
 '總統',
 '希望',
 '2018',
 '進行',
 '包括',
 '市場',
 '目前',
 '營收',
 '曾',
 '約',
 '中心',
 '11',
 '發展',
 '新台幣',
 '未來',
 '計畫',
 '國際',
 '認為',
 '相關',
 '民眾',
 '歲',
 '萬',
 '透過',
 '工作',
 '經濟',
 '項',
 '蘇貞昌',
 '2019',
 '技術',
 '問題',
 '高',
 '活動',
 '人員',
 '點',
 '成長',
 '藝術',
 '台北',
 '媒體',
 '影響',
 '香港',
 '持續',
 '時間',
 '產業',
 '調查',
 '大學',
 '提供',
 '服務',
 '歷史',
 '家',
 '做',
 '30',
 '科技',
 '演出',
 '蘋果',
 '全球',
 '種',
 '產品',
 '政策',
 '部分',
 '未',
 '宣布',
 '萬元',
 '日本',
 '合作',
 '季',
 '要求',
 '提出',
 '北京',
 '20',
 '使用',
 '陳',
 '好',
 '發現',
 '場',
 '舉辦',
 '表現',
 '社會',
 '警方',
 '電影',
 '舉行',
 '重要',
 '高雄',
 '首',
 '賴清德',
 '人士',
 '接受',
 '主要',
 '支持',
 '大陸',
 '遭',
 '委員會',
 '地方',
 '已經',
 '過去',
 '完成',
 '世界',
 '些',
 '內容',
 '手機',
 '機會',
 '發生',
 '部',
 '擔任',
 '推出',
 '投資',
 '處理',
 '中央',
 '代表',
 '天',
 '昨天',
 '是否',
 '立委',
 '選舉',
 '增加',
 '音樂',
 '0',
 '規定',
 '協助',
 '成為',
 '加拿大',
 '最後',
 '業者',
 '超過',
 '創作',
 '台中',
 '15',
 '團

In [351]:
def remove_mix(lab_fwords={}):

    dic3={}
    for lab in lab_fwords.keys():   
        word_content2 = lab_fwords[lab].copy()
#        if fword_value == "dic":
#            word_content2 = list(word_content2.items())
        for lab2 in lab_fwords.keys():    
            if lab != lab2:
#               if fword_value == "items":
                word_content = [w for w in lab_fwords[lab] if (w[0] in [a[0] for a in lab_fwords[lab2]])]
                for w in word_content:
                    try:
                        word_content2.remove(w)                  
                    except ValueError:
                        pass
                        
                        
#            if fword_value == "items":
            dic3[lab] = word_content2 


    return dic3

In [352]:
fword_items = remove_mix(fwords_tags_items)

In [356]:

allfword = [i[0] for v in fword_items.values() for i in v]

In [344]:
www

NameError: name 'www' is not defined

In [357]:
import random
random.shuffle(books)

In [358]:
def pos_fwords(article,top=200):

    features={}
    fwords = get_fwords(article,top)
    for w in fwords :
        w = w[0]
        if w in [i[0] for i in [t for v in fword_items.values() for t in v]]:
            features[w] = True
    return features

In [359]:
def get_features(dic={},top=200):

    article = list(dic.keys())[0]
    labname = list(dic.values())[0]
    features = pos_fwords(article,top)

    if len(list(features.keys())) < 2:
        feature_item = [{},labname]
    else:
        feature_item = [features,labname]

    return feature_item

In [360]:
def get_featuresets(books=books,top=200):
    txt_featuresets = []
    for book in books:

        txt_featureset = get_features(book,top=top)
        txt_featuresets.append(txt_featureset)
    return  txt_featuresets

In [361]:
txt_featuresets = get_featuresets()

KeyboardInterrupt: 

In [ ]:
txt_featuresets

In [ ]:
txt_featuresets = txt_featuresets.copy()
size = int(len(txt_featuresets)*0.7)
train_set, test_set = txt_featuresets[:size], txt_featuresets[size:]
txt_classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(txt_classifier, test_set)

In [ ]:
[i[0]  for v in fwords_tags_items.values() for i in v]

In [ ]:
fwords_tags_items

In [ ]:
from collections import Counter
def get_top(article,top=5,get=20):
    fwords = get_fwords(article)
    txt_comanwer = txt_classifier.classify(pos_fwords(article))

    labcount = []
    for i in fwords:
        w = i[0]
        for lab in fwords_tags_items.keys():
            if w in [wi[0] for wi in fwords_tags_items[lab]]:
                labcount.append(lab)

    commonlist = Counter(labcount).most_common()
    
#    if commonlist[0][1] - commonlist[1][1] <= 2 and txt_comanwer in [i[0]for i in commonlist[:3]]:
#        most_common = txt_comanwer
#    else :
#        most_common = commonlist[0][0]
#    print(commonlist,txt_comanwer,most_common)
    return [i[1] for i in commonlist]

In [ ]:
get_top(testtxt)

In [ ]:
[(get_top(i[0]),i[1]) for i in [list(book.items())[0] for book in books[:30]]]

In [ ]:
[list(book.keys())[0] for book in books]

In [ ]:
from collections import Counter 
def getVocabularyList(content_list, vocabulary_size):
    allContent_str = ''.join(content_list)
    counter = Counter(allContent_str)
    vocabulary_list = [k[0] for k in counter.most_common(vocabulary_size)]
    return ['PAD'] + vocabulary_list
startTime = time.time()
vocabulary_list = getVocabularyList(content_list, 300)
used_time = time.time() - startTime
print('used time: %.2f seconds' %used_time)

In [ ]:
vocabulary_list = [i[0] for c in content_list for i in c]

In [ ]:
vocabulary_list

In [ ]:
dict([(b, a) for a, b in enumerate(vocabulary_list)])

In [ ]:
test_idlist_list = [content2idList(content) for content in test_content_list]
test_X = kr.preprocessing.sequence.pad_sequences(test_idlist_list, sequence_length)
test_y = labelEncoder.transform(test_label_list)
test_Y = kr.utils.to_categorical(test_y, num_classes)
import random
for i in range(10000):
    selected_index = random.sample(list(range(len(train_y))), k=batch_size)
    batch_X = train_X[selected_index]
    batch_Y = train_Y[selected_index]
    session.run(train, {X_holder:batch_X, Y_holder:batch_Y})
    step = i + 1 
    if step % 50 == 0:
        selected_index = random.sample(list(range(len(test_y))), k=200)
        batch_X = test_X[selected_index]
        batch_Y = test_Y[selected_index]
        loss_value, accuracy_value = session.run([loss, accuracy], {X_holder:batch_X, Y_holder:batch_Y})
        print('step:%d loss:%.4f accuracy:%.4f' %(step, loss_value, accuracy_value))

In [ ]:
test_idlist_list

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix

def predictAll(test_X, batch_size=100):
    predict_value_list = []
    for i in range(0, len(test_X), batch_size):
        selected_X = test_X[i: i + batch_size]
        predict_value = session.run(predict_Y, {X_holder:selected_X})
        predict_value_list.extend(predict_value)
    return np.array(predict_value_list)

Y = predictAll(test_X)
y = np.argmax(Y, axis=1)
predict_label_list = labelEncoder.inverse_transform(y)
pd.DataFrame(confusion_matrix(test_label_list, predict_label_list), 
             columns=labelEncoder.classes_,
             index=labelEncoder.classes_ )

In [ ]:
fwords_tags_dic

In [ ]:
get_fwords

In [ ]:
testtxt = list(books[0].keys())[0]

In [ ]:
remove_mix(get_fwords(testtxt))

In [ ]:
def get_featureword(txt):
    fw = [i for i in get_fwords(txt) if i[0] in [i[0] for v in fwords_tags_items.values() for i in v]]
    return fw

In [ ]:
get_featureword(testtxt)

In [450]:
import random
random.shuffle(books)
content_list = [list(book.keys())[0] for book in books]
label_list = [list(book.values())[0] for book in books]

In [451]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(content_list, label_list)
train_content_list = train_X
train_label_list = train_y
test_content_list = test_X
test_label_list = test_y

In [504]:
import time
startTime = time.time()
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(content_list, label_list)
train_content_list = train_X
train_label_list = train_y
test_content_list = test_X
test_label_list = test_y
used_time = time.time() - startTime
print('train_test_split used time : %.2f seconds' %used_time)
#vocabulary_size = len(vocabulary_list)  # 词汇表达小
vocabulary_size = len(allfword)

sequence_length = 200#600  # 序列长度
embedding_size = 64  # 词向量维度
num_filters = 256  # 卷积核数目
filter_size = 5  # 卷积核尺寸
num_fc_units = 128  # 全连接层神经元
dropout_keep_probability = 0.5  # dropout保留比例
learning_rate = 1e-3  # 学习率
batch_size = 40#64  # 每批训练大小

train_test_split used time : 0.00 seconds


In [505]:
#allfwords = [i[0] for v in fwords_tags_items.values() for i in v]
#Counter(allfword).keys()

In [506]:
#vocabulary_list = [i[0] for i in get_fwords(allrawword,4000)]
vocabulary_list = [i for i in Counter(allfword).keys()]

In [507]:
vocabulary_list[:10]

['一國兩制', '中共', '上海', '方面', '統一', '聯合', '北京方面', '官方', '談話', '泛民']

In [508]:
word2id_dict = dict([(d,idx+1) for idx,d in enumerate(vocabulary_list)])
#word2id_dict = dict([(d,idx+1) for idx,d in enumerate(allfword)])

word2id_dict[""] = 0

In [509]:
content2idList = lambda content : [word2id_dict[word] for word in content if word in word2id_dict]

In [510]:
train_idlist_list = [content2idList(content) for content in train_content_list]
used_time = time.time() - startTime
print('content2idList used time : %.2f seconds' %used_time)
import numpy as np
num_classes = np.unique(label_list).shape[0]
import tensorflow.contrib.keras as kr
train_X = kr.preprocessing.sequence.pad_sequences(train_idlist_list, sequence_length)

from sklearn.preprocessing import LabelEncoder
labelEncoder = LabelEncoder()
train_y = labelEncoder.fit_transform(train_label_list)
train_Y = kr.utils.to_categorical(train_y, num_classes)
import tensorflow as tf
tf.reset_default_graph()
X_holder = tf.placeholder(tf.int32, [None, sequence_length])
Y_holder = tf.placeholder(tf.float32, [None, num_classes])
used_time = time.time() - startTime
print('data preparation used time : %.2f seconds' %used_time)

content2idList used time : 0.78 seconds
data preparation used time : 0.80 seconds


In [511]:
len(train_X[2])

200

In [512]:
embedding = tf.get_variable('embedding', 
                            [vocabulary_size, embedding_size])
embedding_inputs = tf.nn.embedding_lookup(embedding,
                                          X_holder)
conv = tf.layers.conv1d(embedding_inputs,
                        num_filters,
                        filter_size)
max_pooling = tf.reduce_max(conv, 
                            [1])
full_connect = tf.layers.dense(max_pooling,
                               num_fc_units)
full_connect_dropout = tf.contrib.layers.dropout(full_connect, 
                                                 keep_prob=dropout_keep_probability)
full_connect_activate = tf.nn.relu(full_connect_dropout)
softmax_before = tf.layers.dense(full_connect_activate,
                                 num_classes)
predict_Y = tf.nn.softmax(softmax_before)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y_holder,
                                                           logits=softmax_before)
loss = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)
isCorrect = tf.equal(tf.argmax(Y_holder, 1), tf.argmax(predict_Y, 1))
accuracy = tf.reduce_mean(tf.cast(isCorrect, tf.float32))

In [513]:
init = tf.global_variables_initializer()
session = tf.Session()
session.run(init)

In [ ]:
test_idlist_list = [content2idList(content) for content in test_content_list]
test_X = kr.preprocessing.sequence.pad_sequences(test_idlist_list, sequence_length)
test_y = labelEncoder.transform(test_label_list)
test_Y = kr.utils.to_categorical(test_y, num_classes)
import random
for i in range(20000):
    selected_index = random.sample(list(range(len(train_y))), k=batch_size)
    batch_X = train_X[selected_index]
    batch_Y = train_Y[selected_index]
    session.run(train, {X_holder:batch_X, Y_holder:batch_Y})
    step = i + 1 
    if step % 100 == 0:
        selected_index = random.sample(list(range(len(test_y))), k=200)
        batch_X = test_X[selected_index]
        batch_Y = test_Y[selected_index]
        loss_value, accuracy_value = session.run([loss, accuracy], {X_holder:batch_X, Y_holder:batch_Y})
        print('step:%d loss:%.4f accuracy:%.4f' %(step, loss_value, accuracy_value))

step:100 loss:1.7996 accuracy:0.3900
step:200 loss:1.5697 accuracy:0.4150
step:300 loss:1.3952 accuracy:0.5950
step:400 loss:1.6083 accuracy:0.5550
step:500 loss:1.6454 accuracy:0.5400
step:600 loss:1.6101 accuracy:0.6100
step:700 loss:1.5938 accuracy:0.5900
step:800 loss:2.0217 accuracy:0.5200


In [ ]:
id2word_dict = dict([(i[1],i[0]) for i in word2id_dict.items()])
[id2word_dict[w] for w in test_X[2]]

In [ ]:
id2word_dict[23]

In [ ]:
word2id_dict

In [ ]:
content_list

In [ ]:
test_X[40]